
# Gabarito – TP2: Delivery & Ratings (CSV → Pandas → POO → Módulos → Exportação)

**Observação:** neste gabarito, adotamos a convenção de que **a avaliação (`avaliacao`) é do item** (prato/bebida).  
A **média do restaurante** é calculada **a partir das médias dos seus itens**.

**Caminho do CSV (relativo):** `../data/pedidos_e_avaliacoes.csv`


In [ ]:

# Imports e caminho padrão
import os
import json
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple, Union

import pandas as pd
import numpy as np

CSV_PATH = "../data/pedidos_e_avaliacoes.csv"



## Exercício 1 – Leitura e inspeção (Enunciado)
Leia o arquivo `pedidos_e_avaliacoes.csv` e apresente:
- quantidade de linhas e colunas;
- tipos observados para cada coluna;
- uma amostra de três registros.


In [ ]:

# Gabarito Ex. 1
df = pd.read_csv(CSV_PATH, encoding="utf-8")
display({"dimensoes": df.shape})
display(df.dtypes)
display(df.head(3))



## Exercício 2 – Limpeza mínima (Enunciado)
Aplique as regras:
- eliminar registros com **avaliação** fora de 1 a 5 ou ausente;
- eliminar registros com **preço** ausente ou menor ou igual a zero;
- padronizar a **categoria** em minúsculas;
- remover espaços extras em nomes de cliente e restaurante;
- informar quantos registros foram removidos e o total final.


In [ ]:

# Gabarito Ex. 2
original = len(df)

df2 = df.copy()
# Normalizações simples
df2["categoria"] = df2["categoria"].astype(str).str.strip().str.lower()
df2["nome_cliente"] = df2["nome_cliente"].astype(str).str.strip()
df2["nome_restaurante"] = df2["nome_restaurante"].astype(str).str.strip()

# Conversões de tipo (defensivas)
df2["avaliacao"] = pd.to_numeric(df2["avaliacao"], errors="coerce").astype("Int64")
df2["preco"] = pd.to_numeric(df2["preco"], errors="coerce")

# Filtros
mask_rating_valida = df2["avaliacao"].between(1, 5, inclusive="both")
mask_preco_valido = df2["preco"].notna() & (df2["preco"] > 0)
df2 = df2[mask_rating_valida & mask_preco_valido].copy()

removidos = original - len(df2)
display({"removidos": removidos, "total_final": len(df2)})



## Exercício 3 – Normalizações simples (Enunciado)
- remover duplicatas exatas;
- criar a coluna **faixa_preco** com três categorias (“baixo”, “médio”, “alto”), a partir de limites definidos por você;
- criar a coluna **tamanho_nome_item**, com o número de caracteres do nome do item.


In [ ]:

# Gabarito Ex. 3
df3 = df2.drop_duplicates().copy()

# Definição de limites (exemplo de gabarito)
lim_baixo = 15.0
lim_alto = 30.0

bins = [-np.inf, lim_baixo, lim_alto, np.inf]
labels = ["baixo", "médio", "alto"]
df3["faixa_preco"] = pd.cut(df3["preco"], bins=bins, labels=labels, right=True)

df3["tamanho_nome_item"] = df3["item"].astype(str).str.len()

display(df3.head(5)[["item","preco","faixa_preco","tamanho_nome_item"]])



## Exercício 4 – Checagens rápidas (Enunciado)
- apresentar a quantidade de **clientes distintos**, **restaurantes distintos** e **itens distintos**;
- mostrar a contagem de registros por **categoria** e por **faixa_preco**.


In [ ]:

# Gabarito Ex. 4
clientes_distintos = df3["nome_cliente"].nunique()
# Considera-se restaurante distinto por (nome_restaurante, bairro)
restaurantes_distintos = df3[["nome_restaurante","bairro"]].drop_duplicates().shape[0]
itens_distintos = df3["item"].nunique()

contagem_categoria = df3["categoria"].value_counts().rename_axis("categoria").reset_index(name="qtd")
contagem_faixa = df3["faixa_preco"].value_counts(dropna=False).rename_axis("faixa_preco").reset_index(name="qtd")

display({"clientes_distintos": clientes_distintos,
         "restaurantes_distintos": restaurantes_distintos,
         "itens_distintos": itens_distintos})
display(contagem_categoria)
display(contagem_faixa)



## Exercício 5 – Criação das classes (Enunciado)
Crie as classes **Prato**, **Bebida**, **Restaurante** e **Cliente** com os atributos e métodos básicos.


In [ ]:

# Gabarito Ex. 5 – Implementação das classes

class Prato:
    def __init__(self, nome: str, preco: float):
        if not nome.strip():
            raise ValueError("Nome do prato não pode ser vazio.")
        if preco <= 0:
            raise ValueError("Preço do prato deve ser > 0.")
        self.nome = nome.strip()
        self.preco = float(preco)
        self.notas: List[int] = []

    def descricao(self) -> str:
        return f"PRATO: {self.nome} — R$ {self.preco:.2f}"

    def receber_avaliacao(self, nota: int) -> None:
        if not (1 <= int(nota) <= 5):
            raise ValueError("Nota deve estar entre 1 e 5.")
        self.notas.append(int(nota))

    def media_avaliacao(self) -> Optional[float]:
        return float(np.mean(self.notas)) if self.notas else None


class Bebida:
    def __init__(self, nome: str, preco: float):
        if not nome.strip():
            raise ValueError("Nome da bebida não pode ser vazio.")
        if preco <= 0:
            raise ValueError("Preço da bebida deve ser > 0.")
        self.nome = nome.strip()
        self.preco = float(preco)
        self.notas: List[int] = []

    def descricao(self) -> str:
        return f"BEBIDA: {self.nome} — R$ {self.preco:.2f}"

    def receber_avaliacao(self, nota: int) -> None:
        if not (1 <= int(nota) <= 5):
            raise ValueError("Nota deve estar entre 1 e 5.")
        self.notas.append(int(nota))

    def media_avaliacao(self) -> Optional[float]:
        return float(np.mean(self.notas)) if self.notas else None


class Restaurante:
    def __init__(self, nome: str, bairro: str):
        if not nome.strip() or not bairro.strip():
            raise ValueError("Nome e bairro do restaurante não podem ser vazios.")
        self.nome = nome.strip()
        self.bairro = bairro.strip()
        self.pratos: Dict[str, Prato] = {}
        self.bebidas: Dict[str, Bebida] = {}
        self.notas: List[int] = []  # opcional, caso queira avaliações diretas

    def adicionar_item(self, item: Union[Prato, Bebida]) -> None:
        if isinstance(item, Prato):
            if item.nome in self.pratos:
                return  # evita duplicação
            self.pratos[item.nome] = item
        elif isinstance(item, Bebida):
            if item.nome in self.bebidas:
                return
            self.bebidas[item.nome] = item
        else:
            raise TypeError("Item precisa ser Prato ou Bebida.")

    def buscar_item_por_nome(self, nome: str) -> Optional[Union[Prato, Bebida]]:
        if nome in self.pratos:
            return self.pratos[nome]
        if nome in self.bebidas:
            return self.bebidas[nome]
        return None

    def listar_itens_por_categoria(self, categoria: str) -> List[str]:
        categoria = categoria.strip().lower()
        if categoria == "prato":
            return list(self.pratos.keys())
        elif categoria == "bebida":
            return list(self.bebidas.keys())
        return []

    def total_itens(self) -> int:
        return len(self.pratos) + len(self.bebidas)

    def media_avaliacao(self) -> Optional[float]:
        # média das médias dos itens (se existirem)
        medias = []
        medias += [p.media_avaliacao() for p in self.pratos.values() if p.media_avaliacao() is not None]
        medias += [b.media_avaliacao() for b in self.bebidas.values() if b.media_avaliacao() is not None]
        return float(np.mean(medias)) if medias else None


class Cliente:
    def __init__(self, nome: str):
        if not nome.strip():
            raise ValueError("Nome do cliente não pode ser vazio.")
        self.nome = nome.strip()
        # favoritos: set de tuplas (nome_restaurante, bairro)
        self.favoritos: set[Tuple[str, str]] = set()

    def favoritar_restaurante(self, restaurante: Restaurante) -> None:
        self.favoritos.add((restaurante.nome, restaurante.bairro))

    def desfavoritar_restaurante(self, restaurante: Restaurante) -> None:
        self.favoritos.discard((restaurante.nome, restaurante.bairro))

    def listar_favoritos(self) -> List[Tuple[str, str]]:
        return sorted(list(self.favoritos), key=lambda x: (x[0].lower(), x[1].lower()))



## Exercício 6 – Métodos e validações (Gabarito de demonstração)
Exemplo de uso dos métodos, duplicações, listagem por categoria e favoritos.


In [ ]:

# Demonstração rápida (pode ajustar/expandir)
r = Restaurante("Borogodó Lanches", "Centro")
p1 = Prato("Hambúrguer da Casa", 28.90)
b1 = Bebida("Suco de Laranja", 8.50)
r.adicionar_item(p1)
r.adicionar_item(b1)
r.adicionar_item(p1)  # duplicado (ignorado)

c = Cliente("Luis")
c.favoritar_restaurante(r)
c.favoritar_restaurante(r)  # duplicado (idempotente)
fav_antes = c.listar_favoritos()

lista_pratos = r.listar_itens_por_categoria("prato")
lista_bebidas = r.listar_itens_por_categoria("bebida")

display({"favoritos": fav_antes,
         "pratos": lista_pratos,
         "bebidas": lista_bebidas})



## Exercício 7 – Avaliações (parte 1) (Enunciado)
Adicione avaliações de 1 a 5 em Restaurante, Prato e Bebida.  
Cada item armazena notas e calcula sua média individual.  
O restaurante calcula sua média a partir das médias dos seus itens.


In [ ]:

# Gabarito Ex. 7 – Demonstração
p1.receber_avaliacao(5)
p1.receber_avaliacao(4)
b1.receber_avaliacao(5)

display({"media_prato": p1.media_avaliacao(),
         "media_bebida": b1.media_avaliacao(),
         "media_restaurante": r.media_avaliacao()})



## Exercício 8 – Integração dos dados nas classes (Enunciado)
Usando o conjunto de dados limpo, identificar restaurantes únicos (nome, bairro) e criar uma lista de objetos Restaurante.


In [ ]:

# Gabarito Ex. 8
restaurantes_unicos = (df3[["nome_restaurante","bairro"]]
                       .drop_duplicates()
                       .reset_index(drop=True))

restaurantes = []
for _, row in restaurantes_unicos.iterrows():
    restaurantes.append(Restaurante(row["nome_restaurante"], row["bairro"]))

len_restaurantes = len(restaurantes)
preview = [(restaurantes[i].nome, restaurantes[i].bairro) for i in range(min(5, len_restaurantes))]
display({"total_restaurantes": len_restaurantes, "exemplo": preview})



## Exercício 9 – Consolidação de avaliações (Enunciado)
Simular (ou aplicar) avaliações: garantir que cada restaurante e alguns itens recebam notas.  
Em seguida, exibir as médias finais de cada restaurante e de cada item.


In [ ]:

# Gabarito Ex. 9 – Alimentar itens e avaliações a partir do df3
# Estratégia: para cada restaurante, crie itens únicos (item, categoria) com um preço médio,
# depois alimente avaliações conforme 'avaliacao' do df3.

# Índice auxiliar para acessar objetos Restaurante por (nome, bairro)
map_rest: Dict[Tuple[str,str], Restaurante] = {(r.nome, r.bairro): r for r in restaurantes}

# Criar estruturas de itens por restaurante
grupo = df3.groupby(["nome_restaurante","bairro","item","categoria"], as_index=False).agg(
    preco_medio=("preco","mean"),
    qtd=("id_pedido","count"),
    media_avaliacao_item=("avaliacao","mean"),
)

for _, row in grupo.iterrows():
    chave_rest = (row["nome_restaurante"], row["bairro"])
    rest_obj = map_rest[chave_rest]
    if row["categoria"] == "prato":
        item_obj = rest_obj.buscar_item_por_nome(row["item"])
        if item_obj is None or not isinstance(item_obj, Prato):
            item_obj = Prato(row["item"], float(row["preco_medio"]))
            rest_obj.adicionar_item(item_obj)
    else:
        item_obj = rest_obj.buscar_item_por_nome(row["item"])
        if item_obj is None or not isinstance(item_obj, Bebida):
            item_obj = Bebida(row["item"], float(row["preco_medio"]))
            rest_obj.adicionar_item(item_obj)

# Alimentar avaliações linha a linha (amostra para reduzir custo)
amostra = df3.sample(min(500, len(df3)), random_state=42)
for _, row in amostra.iterrows():
    rest = map_rest[(row["nome_restaurante"], row["bairro"])]
    it = rest.buscar_item_por_nome(row["item"])
    if it is not None:
        it.receber_avaliacao(int(row["avaliacao"]))

# Consolidar médias
resumo_rest = []
resumo_itens = []

for rest in restaurantes:
    resumo_rest.append({
        "restaurante": rest.nome,
        "bairro": rest.bairro,
        "media": rest.media_avaliacao(),
        "qtd_itens": rest.total_itens()
    })
    for it in list(rest.pratos.values()) + list(rest.bebidas.values()):
        resumo_itens.append({
            "restaurante": rest.nome,
            "bairro": rest.bairro,
            "item": it.nome,
            "categoria": "prato" if isinstance(it, Prato) else "bebida",
            "preco": it.preco,
            "media": it.media_avaliacao(),
            "qtd_avaliacoes": len(it.notas)
        })

df_rest_medias = pd.DataFrame(resumo_rest)
df_itens_medias = pd.DataFrame(resumo_itens)

display(df_rest_medias.head(10))
display(df_itens_medias.head(10))



## Exercício 10 – Rankings (Enunciado)
Gerar dois rankings simples:
- Top 3 restaurantes por média (desempate: maior número de avaliações dos itens).
- Top 3 itens por média (desempate: menor preço).


In [ ]:

# Gabarito Ex. 10
# Para restaurantes, aproximamos "número de avaliações" pela soma das avaliações dos itens.
def _calc_qtd_avaliacoes_rest(row):
    # recomputa a partir de df_itens_medias
    mask = (df_itens_medias["restaurante"] == row["restaurante"]) & (df_itens_medias["bairro"] == row["bairro"])
    return int(df_itens_medias.loc[mask, "qtd_avaliacoes"].sum())

df_rest_rank = df_rest_medias.copy()
df_rest_rank["qtd_avaliacoes_itens"] = df_rest_rank.apply(_calc_qtd_avaliacoes_rest, axis=1)
df_rest_rank = df_rest_rank.dropna(subset=["media"])
df_rest_rank = df_rest_rank.sort_values(by=["media","qtd_avaliacoes_itens"], ascending=[False, False]).head(3)

df_itens_rank = df_itens_medias.dropna(subset=["media"]).sort_values(
    by=["media","preco"], ascending=[False, True]
).head(3)

display({"top3_restaurantes": df_rest_rank.reset_index(drop=True),
         "top3_itens": df_itens_rank.reset_index(drop=True)})



## Exercício 11 – Reorganização em módulos (Enunciado)
Crie uma pasta do projeto e divida o código em quatro módulos:
- `restaurante.py` — classe Restaurante
- `cliente.py` — classe Cliente
- `produtos.py` — classes Prato e Bebida
- `main.py` — arquivo principal

Gabarito (orientação): copie as classes deste notebook para os respectivos arquivos.



## Exercício 12 – Exportação de resultados (Enunciado)
Exportar os principais resultados para arquivos CSV/JSON na pasta `../dados`:
- Catálogo de restaurantes (nome, bairro, lista de itens com preços);
- Resumo por restaurante (nome, bairro, número de itens e média);
- Rankings finais (Top N restaurantes e Top N itens).


In [ ]:

# Gabarito Ex. 12
saida_dir = "../dados"
os.makedirs(saida_dir, exist_ok=True)

# Catálogo por restaurante (JSON)
catalogo = []
for rest in restaurantes:
    catalogo.append({
        "restaurante": rest.nome,
        "bairro": rest.bairro,
        "pratos": [{"nome": p.nome, "preco": p.preco} for p in rest.pratos.values()],
        "bebidas": [{"nome": b.nome, "preco": b.preco} for b in rest.bebidas.values()],
    })
with open(os.path.join(saida_dir, "catalogo_restaurantes.json"), "w", encoding="utf-8") as f:
    json.dump(catalogo, f, ensure_ascii=False, indent=2)

# Resumo por restaurante (CSV)
df_rest_medias.to_csv(os.path.join(saida_dir, "resumo_restaurantes.csv"), index=False, encoding="utf-8")

# Rankings (CSV)
# Reaproveita df_rest_rank e df_itens_rank do Ex. 10
df_rest_rank.to_csv(os.path.join(saida_dir, "ranking_restaurantes.csv"), index=False, encoding="utf-8")
df_itens_rank.to_csv(os.path.join(saida_dir, "ranking_itens.csv"), index=False, encoding="utf-8")

print("Arquivos exportados em:", os.path.abspath(saida_dir))
